# problema di assegnamento

Un sistema multiprocessore con 3 CPU (A,B e C) deve eseguire 3 processi (1,2 e 3). 

Ogni processo richiede 10 ms di CPU time e può essere frazionato tra le CPU ma ogni CPU è disponibile per una quantità di tempo al massimo pari a 10 ms.

Date le differenti caratteristiche delle CPU, i costi unitari di processamento dipendono dalla coppia processo-CPU (vedi tabella).

![tab](./tab.png)

Come assegnare i processi alle CPU in modo da minimizzare il costo totale?


## formulazione generale

Due insiemi A e B sono costituiti ognuno da n elementi.

Ogni elemento di A deve essere assegnato a un elemento di B (o frazionato tra più elementi di B);

a ogni elemento di B può essere assegnato al più un elemento di A (o frazioni di elementi di A che sommano 1).

Assegnare l’elemento $i\in A$ all’elemento $j\in B$ costa $c_{ij}$.

Qual è l’assegnamento di costo minimo?


## modello parametrico

![param1](param1.png)

![param2](param2.png)

### caso in cui gli elementi di A non siano frazionabili

![param3](param3.png)

![param4](param4.png)

![param5](param5.png)

![param6](param6.png)

In [6]:
from pyomo.environ import *
model = ConcreteModel('problema di trasporto')

i = [ 'pr1', 'pr2', 'pr3' ]

j = [ 'cpuA', 'cpuB', 'cpuC' ]

c = [[ 7, 11,  9],
     [ 8, 10, 12],
     [13, 12,  8]]

model.i = Set(initialize=i)
model.j = Set(initialize=j)

c_dict = {}
for i, mi in enumerate(model.i):
    for j, mj in enumerate(model.j):
        c_dict[mi, mj] = c[i][j]

model.c = Param(model.i, model.j, initialize=c_dict)

model.x = Var(model.i, model.j, domain=Boolean, initialize=0)

obj_expr = sum(sum(model.c[i, j]*model.x[i, j] for j in model.j) for i in model.i)
model.cost = Objective(expr = obj_expr, sense=minimize)

model.constraints = ConstraintList()
for i in model.i:
    model.constraints.add(expr = sum(model.x[i, j] for j in model.j) == 1)

for j in model.j:
    model.constraints.add(expr = sum(model.x[i, j] for i in model.i) == 1)

SolverFactory('glpk').solve(model)
model.display()

Model problema di trasporto

  Variables:
    x : Size=9, Index=x_index
        Key             : Lower : Value : Upper : Fixed : Stale : Domain
        ('pr1', 'cpuA') :     0 :   1.0 :     1 : False : False : Boolean
        ('pr1', 'cpuB') :     0 :   0.0 :     1 : False : False : Boolean
        ('pr1', 'cpuC') :     0 :   0.0 :     1 : False : False : Boolean
        ('pr2', 'cpuA') :     0 :   0.0 :     1 : False : False : Boolean
        ('pr2', 'cpuB') :     0 :   1.0 :     1 : False : False : Boolean
        ('pr2', 'cpuC') :     0 :   0.0 :     1 : False : False : Boolean
        ('pr3', 'cpuA') :     0 :   0.0 :     1 : False : False : Boolean
        ('pr3', 'cpuB') :     0 :   0.0 :     1 : False : False : Boolean
        ('pr3', 'cpuC') :     0 :   1.0 :     1 : False : False : Boolean

  Objectives:
    cost : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  25.0

  Constraints:
    constraints : Size=6
        Key : Lower : Body : U